In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


In [ ]:
data = {
    'AQI': [75, 90, 85, 60, 95, 88, 70, 80],
    'Temperature': [22, 25, 23, 20, 26, 24, 21, 23],
    'Humidity': [55, 60, 58, 65, 52, 57, 59, 61],
    'CO2': [400, 420, 415, 390, 430, 410, 395, 405],
    'PM2.5': [35, 40, 38, 30, 45, 37, 33, 39]
}
df = pd.DataFrame(data)


In [ ]:
print("Missing Values:\n", df.isnull().sum())


In [ ]:
df.loc[2, 'Humidity'] = np.nan  # Simulate missing value

# Fill with column mean
df['Humidity'].fillna(df['Humidity'].mean(), inplace=True)


In [ ]:
print("\nCorrelation Matrix:\n", correlation_matrix.round(2))


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Heatmap of Environmental Metrics')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Coolwarm Heatmap of Environmental Correlations')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='viridis',
    linewidths=1,
    linecolor='black'
)
plt.title('Styled Environmental Heatmap (Viridis)')
plt.show()


In [ ]:
# Create a mask for any missing values in correlation matrix
mask = correlation_matrix.isnull()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', mask=mask)
plt.title('Heatmap with Masked Missing Data')
plt.show()


In [ ]:
# Define a custom diverging color map
custom_cmap = LinearSegmentedColormap.from_list("env_cmap", ["#d73027", "#ffffbf", "#1a9850"])

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap=custom_cmap, linewidths=0.5)
plt.title('Environmental Correlation Heatmap (Custom Colors)')
plt.show()
